In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark import SparkContext
sc =SparkContext(master="local", appName="New Spark Context")

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [5]:
from pyspark.sql.functions import *
import pandas as pd


### Import dữ liệu sau processed

In [6]:
data = spark.read.csv("Reviews_preprocessed.csv",header=True,encoding="UTF-8")
data.show(5)

+------------+--------------------+--------------+
|IDRestaurant|         Comment_new|rating_encoded|
+------------+--------------------+--------------+
|           1|          chiên sống|             1|
|           1|                ngon|             1|
|           1|                thừa|             0|
|           1|kêu đói háo_hức t...|             1|
|           1|           tặng uống|             2|
+------------+--------------------+--------------+
only showing top 5 rows



In [7]:
data = data.select(["Comment_new", "rating_encoded"])

In [8]:
data.count()

29917

In [9]:
data.printSchema()

root
 |-- Comment_new: string (nullable = true)
 |-- rating_encoded: string (nullable = true)



In [10]:
data.groupby('rating_encoded').count().show()

+--------------+-----+
|rating_encoded|count|
+--------------+-----+
|             0| 3707|
|             1| 4116|
|             2|22094|
+--------------+-----+



In [11]:
data = data.withColumn("rating_encoded", col("rating_encoded").cast("integer"))
data.printSchema()

root
 |-- Comment_new: string (nullable = true)
 |-- rating_encoded: integer (nullable = true)



In [12]:
data = data.withColumn('length',length(data['Comment_new']))
data.show()

+--------------------+--------------+------+
|         Comment_new|rating_encoded|length|
+--------------------+--------------+------+
|          chiên sống|             1|    10|
|                ngon|             1|     4|
|                thừa|             0|     4|
|kêu đói háo_hức t...|             1|    78|
|           tặng uống|             2|     9|
|thề lắm tức_giận ...|             0|   113|
|thử hơi kén không...|             2|    54|
|kết_hợp không_thấ...|             0|    33|
|  dở ăn_tây tây ngon|             1|    18|
|                cười|             0|     4|
|khó_chịu mặt khôn...|             1|    39|
|ghé đakao hướng t...|             0|   172|
|        bảo kiểm_trà|             0|    12|
|đi kẹp bánh_mì kẹ...|             2|    69|
|chú_ý toàn_ức khô...|             0|    23|
|chán ngon buger l...|             2|    41|
|miếng không_nổi đ...|             0|    31|
|xuyên kiểm_trà đồ...|             0|    43|
|thức_ăn nhẵn_mặt ...|             2|   102|
|         

In [13]:
data.groupby('rating_encoded').mean().show()

+--------------+-------------------+------------------+
|rating_encoded|avg(rating_encoded)|       avg(length)|
+--------------+-------------------+------------------+
|             1|                1.0|63.505344995140916|
|             2|                2.0| 77.01317099665067|
|             0|                0.0| 73.82222821688697|
+--------------+-------------------+------------------+



### Feature Engineering

In [14]:
from pyspark.ml.feature import  CountVectorizer,IDF, Tokenizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create stages for the Pipeline

tokenizer = Tokenizer(inputCol="Comment_new", outputCol="token_text")
count_vec = CountVectorizer(inputCol='token_text', outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
clean_up = VectorAssembler(inputCols=["tf_idf", "length"], outputCol="features")

In [15]:
from pyspark.ml import Pipeline

# Define the Pipeline
data_prep_pipe = Pipeline(stages=[tokenizer, count_vec, idf, clean_up])

# Fit and transform the Pipeline
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

# Show the transformed DataFrame
clean_data.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
clean_data.count()

29917

### Split dữ liệu

In [17]:

clean_data = clean_data.select(['rating_encoded','features']).withColumnRenamed("rating_encoded", "label")

training,testing = clean_data.randomSplit([0.7,0.3])

In [18]:
training.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(27643,[0,1,2,3,4...|
|    0|(27643,[0,1,2,3,5...|
|    0|(27643,[0,1,2,3,5...|
|    0|(27643,[0,1,2,3,5...|
|    0|(27643,[0,1,2,3,6...|
+-----+--------------------+
only showing top 5 rows



In [19]:
testing.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(27643,[0,1,2,3,5...|
|    0|(27643,[0,1,2,3,8...|
|    0|(27643,[0,1,2,5,8...|
|    0|(27643,[0,1,2,5,1...|
|    0|(27643,[0,1,2,5,1...|
+-----+--------------------+
only showing top 5 rows



### Chọn model NaiveBays

In [20]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

predictor = nb.fit(training)
test_results = predictor.transform(testing)
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(27643,[0,1,2,3,5...|[-773.09356550274...|[1.0,2.0884742583...|       0.0|
|    0|(27643,[0,1,2,3,8...|[-294.88512680423...|[4.68537191087503...|       2.0|
|    0|(27643,[0,1,2,5,8...|[-1635.6711734652...|[0.99999999999792...|       0.0|
|    0|(27643,[0,1,2,5,1...|[-1348.7644449329...|[2.40483639865356...|       1.0|
|    0|(27643,[0,1,2,5,1...|[-1811.1729554060...|[1.0,2.8422235758...|       0.0|
|    0|(27643,[0,1,2,5,2...|[-719.97220114174...|[1.52255118442041...|       2.0|
|    0|(27643,[0,1,2,6,1...|[-1299.1963062929...|[3.16816797611031...|       1.0|
|    0|(27643,[0,1,2,7,1...|[-652.79403785723...|[2.97620614333949...|       1.0|
|    0|(27643,[0,1,2,18,...|[-356.80249615426...|[0.97723098884360...|       0.0|
|    0|(27643,[0

In [21]:
test_results.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    2|       1.0| 1255|
|    1|       0.0|  272|
|    0|       2.0|  141|
|    0|       0.0|  663|
|    1|       2.0|  523|
|    2|       2.0| 5095|
|    1|       1.0|  379|
|    2|       0.0|  308|
|    0|       1.0|  323|
+-----+----------+-----+



In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create a MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

# Calculate precision
precision = evaluator.evaluate(test_results)

# Reset the evaluator to calculate recall
evaluator.setMetricName("weightedRecall")

# Calculate recall
recall = evaluator.evaluate(test_results)

# Reset the evaluator to calculate f1-score
evaluator.setMetricName("f1")

# Calculate f1-score
f1_score = evaluator.evaluate(test_results)

# Print the classification report
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1_score))

Precision: 0.75
Recall: 0.69
F1-score: 0.71


Nhận xét:
- Precision: 75% các bài đánh giá được mô hình dự đoán là "negative", "neutral" hoặc "positive" là chính xác.

- Recall: 69% các bài đánh giá thực sự là "negative", "neutral" hoặc "positive" được mô hình dự đoán chính xác.

- F1-score: cho thấy mô hình có khả năng dự đoán khá tốt các bài đánh giá với độ precision và recall tốt

Tuy nhiên, vẫn còn một số điểm cần cải thiện:

- Recall: Recall thấp hơn Precision cho thấy mô hình có thể bỏ sót một số bài đánh giá, đặc biệt là những bài có mức độ sentimal neutral hoặc khó phân biệt.

- F1-score: F1-score tuy ở mức khá tốt nhưng vẫn có thể được nâng cao để tăng hiệu quả phân loại.

### Propose model : Chọn model RandomForestClassifier

In [26]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier


# RandomForestClassifier model
rf = RandomForestClassifier(featuresCol="features", labelCol="label")

# Train the model
model = rf.fit(training)

# Make predictions on the testing data
predictions = model.transform(testing)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)

Accuracy: 0.7431632994753878


In [27]:
# Calculate precision
precision = evaluator.evaluate(predictions)

# Reset the evaluator to calculate recall
evaluator.setMetricName("weightedRecall")

# Calculate recall
recall = evaluator.evaluate(predictions)

# Reset the evaluator to calculate f1-score
evaluator.setMetricName("f1")

# Calculate f1-score
f1_score = evaluator.evaluate(predictions)

# Print the classification report
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1_score))

Precision: 0.74
Recall: 0.74
F1-score: 0.63


Nhận xét: Giá trị precision đối với model RandomforestClassifier thấp hơn 1 chút so với NavieBays, tuy nhiên Recall và F1-Score cao hơn 
=> Model Randomforest vẫn có thể được sử dụng tốt cho bài toán Sentimental Analyst